In [21]:
import os
import json
import shutil

In [112]:
SOURCE_DATA_DIR = "data/RF_sorted_data"

BCS_OUT_DIR = 'data/RF_sorted_data/BCS'
BCS_YOLO_DIR = 'data/YOLOv8/BCS'

UTYPE_OUT_DIR = 'data/RF_sorted_data/UdderType'
UTYPE_YOLO_DIR = 'data/YOLOv8/UdderType'

CLEFT_OUT_DIR = 'data/RF_sorted_data/Cleft'
CLEFT_YOLO_DIR = 'data/YOLOv8/Cleft'

TSCORE_OUT_DIR = 'data/RF_sorted_data/TScore'
TSCORE_YOLO_DIR = 'data/YOLOv8/TScore'

BREED_OUT_DIR = 'data/RF_sorted_data/CowBreed'
BREED_YOLO_DIR = 'data/YOLOv8/CowBreed'

In [23]:
def convert_coco_to_yolov8(bbox, img_width, img_height):
    x_min, y_min, bbox_width, bbox_height = bbox
    x_center = (x_min + bbox_width / 2) / img_width
    y_center = (y_min + bbox_height / 2) / img_height
    width = bbox_width / img_width
    height = bbox_height / img_height
    return x_center, y_center, width, height

In [55]:
def aggregate_class_folders(source_dir, target_dir, main_class):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir, exist_ok=True)
    for subdir in os.listdir(source_dir):
        if subdir.startswith(main_class) or subdir.endswith(main_class):
            shutil.copytree(os.path.join(source_dir, subdir), os.path.join(target_dir, subdir))
            print(f'Copied {subdir} to {target_dir}')

In [118]:
def reformat_coco_annots(source_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)

    img_dir = os.path.join(output_dir, 'train/images')
    labels_dir = os.path.join(output_dir, 'train/labels')

    if not os.path.exists(labels_dir):
        os.makedirs(labels_dir, exist_ok=True)
    if not os.path.exists(img_dir):
        os.makedirs(img_dir, exist_ok=True)

    img_width, img_height = 640, 640

    for cat_id, cat_dir in enumerate(os.listdir(source_dir)):
        cat_path = os.path.join(source_dir, cat_dir)
        if os.path.isdir(cat_path):
            annotation_file = [f for f in os.listdir(cat_path) if f.endswith('.json')][0]
            annotation_path = os.path.join(cat_path, annotation_file)

            with open(annotation_path, 'r') as f:
                annotations = json.load(f)

            for img_file, annots in annotations.items():
                src_img_path = os.path.join(cat_path, img_file)
                trgt_img_path = os.path.join(img_dir, img_file)

                shutil.copy(src_img_path, trgt_img_path)
                # print(img_file, annots)
                bbox = annots['bbox']
                x_center, y_center, width, height = convert_coco_to_yolov8(bbox, img_width, img_height)

                yolo_annotation = f"{cat_id} {x_center} {y_center} {width} {height}"
                label_file_name = img_file.replace('.jpg', '.txt')
                label_file_path = os.path.join(labels_dir, label_file_name)
                # print(yolo_annotation, label_file_path)
                with open(label_file_path, 'w') as label_file:
                    label_file.write(yolo_annotation + "\n")
        print(f"Successfully converted {cat_dir} to yolov8 annotation format.")
    print(f"Finished conversion of {source_dir} to yolov8 annotation format.")

In [119]:
def compare_file_counts(src_dir, agg_dir, yolo_dir, main_class) -> None:
    init_cnt, post_cnt, yolo_cnt = 0, 0, 0
    for src_subdir in os.listdir(src_dir):
        if src_subdir.startswith(main_class) or src_subdir.endswith(main_class):
            subdir_path = os.path.join(src_dir, src_subdir)
            for file in os.listdir(subdir_path):
                if file.endswith(".jpg"):
                    init_cnt += 1
    for agg_subdir in os.listdir(agg_dir):
        if agg_subdir.startswith(main_class) or agg_subdir.endswith(main_class):
            subdir_path = os.path.join(agg_dir, agg_subdir)
            for file in os.listdir(subdir_path):
                if file.endswith(".jpg"):
                    post_cnt += 1
                    
    for yolo_file in os.listdir(yolo_dir + "/train/images/"):
        if yolo_file.endswith(".jpg"):
            yolo_cnt += 1
            
    print(f"Pre-aggregation: {init_cnt}, Post-aggregation: {post_cnt}, Post-conversion: {yolo_cnt}")

# Conversion of BCS data

In [122]:
aggregate_class_folders(SOURCE_DATA_DIR, BCS_OUT_DIR, main_class='BCS-')

Copied BCS-1.25 to data/RF_sorted_data/BCS
Copied BCS-1 to data/RF_sorted_data/BCS
Copied BCS-4.75 to data/RF_sorted_data/BCS
Copied BCS-4 to data/RF_sorted_data/BCS
Copied BCS-1.5 to data/RF_sorted_data/BCS
Copied BCS-2.75 to data/RF_sorted_data/BCS
Copied BCS-4.25 to data/RF_sorted_data/BCS
Copied BCS-2 to data/RF_sorted_data/BCS
Copied BCS-4.5 to data/RF_sorted_data/BCS
Copied BCS-2.5 to data/RF_sorted_data/BCS
Copied BCS-1.75 to data/RF_sorted_data/BCS
Copied BCS-3 to data/RF_sorted_data/BCS
Copied BCS-3.75 to data/RF_sorted_data/BCS
Copied BCS-2.25 to data/RF_sorted_data/BCS
Copied BCS-3.25 to data/RF_sorted_data/BCS
Copied BCS-3.5 to data/RF_sorted_data/BCS


In [123]:
reformat_coco_annots(BCS_OUT_DIR, BCS_YOLO_DIR)

Successfully converted BCS-1.25 to yolov8 annotation format.
Successfully converted BCS-1 to yolov8 annotation format.
Successfully converted BCS-4.75 to yolov8 annotation format.
Successfully converted BCS-4 to yolov8 annotation format.
Successfully converted BCS-1.5 to yolov8 annotation format.
Successfully converted BCS-2.75 to yolov8 annotation format.
Successfully converted BCS-4.25 to yolov8 annotation format.
Successfully converted BCS-2 to yolov8 annotation format.
Successfully converted BCS-4.5 to yolov8 annotation format.
Successfully converted BCS-2.5 to yolov8 annotation format.
Successfully converted BCS-1.75 to yolov8 annotation format.
Successfully converted BCS-3 to yolov8 annotation format.
Successfully converted BCS-3.75 to yolov8 annotation format.
Successfully converted BCS-2.25 to yolov8 annotation format.
Successfully converted BCS-3.25 to yolov8 annotation format.
Successfully converted BCS-3.5 to yolov8 annotation format.
Finished conversion of data/RF_sorted_da

In [124]:
compare_file_counts(SOURCE_DATA_DIR, BCS_OUT_DIR, BCS_YOLO_DIR, main_class='BCS-')

Pre-aggregation: 100, Post-aggregation: 100, Post-conversion: 100


# Conversion of Type of Udder data

In [125]:
aggregate_class_folders(SOURCE_DATA_DIR, UTYPE_OUT_DIR, main_class='-Udder')

Copied Moderately-Pendulous-Udder to data/RF_sorted_data/UdderType
Copied Compact-Udder to data/RF_sorted_data/UdderType
Copied Pendulous-Udder to data/RF_sorted_data/UdderType
Copied Small-Tight-Udder to data/RF_sorted_data/UdderType


In [126]:
reformat_coco_annots(UTYPE_OUT_DIR, UTYPE_YOLO_DIR)

Successfully converted Moderately-Pendulous-Udder to yolov8 annotation format.
Successfully converted Compact-Udder to yolov8 annotation format.
Successfully converted Pendulous-Udder to yolov8 annotation format.
Successfully converted Small-Tight-Udder to yolov8 annotation format.
Finished conversion of data/RF_sorted_data/UdderType to yolov8 annotation format.


In [127]:
compare_file_counts(SOURCE_DATA_DIR, UTYPE_OUT_DIR, UTYPE_YOLO_DIR, main_class='-Udder')

Pre-aggregation: 84, Post-aggregation: 84, Post-conversion: 84


# Conversion of Cleft data

In [128]:
aggregate_class_folders(SOURCE_DATA_DIR, CLEFT_OUT_DIR, main_class='Clefted')

Copied Moderately-Clefted to data/RF_sorted_data/Cleft
Copied Slightly-Clefted to data/RF_sorted_data/Cleft
Copied Deep-Clefted to data/RF_sorted_data/Cleft


In [129]:
reformat_coco_annots(CLEFT_OUT_DIR, CLEFT_YOLO_DIR)

Successfully converted Moderately-Clefted to yolov8 annotation format.
Successfully converted Slightly-Clefted to yolov8 annotation format.
Successfully converted Deep-Clefted to yolov8 annotation format.
Finished conversion of data/RF_sorted_data/Cleft to yolov8 annotation format.


In [130]:
compare_file_counts(SOURCE_DATA_DIR, CLEFT_OUT_DIR, CLEFT_YOLO_DIR, main_class="Clefted")

Pre-aggregation: 115, Post-aggregation: 115, Post-conversion: 115


# Conversion of Teat Score data

In [131]:
aggregate_class_folders(SOURCE_DATA_DIR, TSCORE_OUT_DIR, main_class="Teat-Score")

Copied Teat-Score-5 to data/RF_sorted_data/TScore
Copied Teat-Score-7 to data/RF_sorted_data/TScore
Copied Teat-Score-3 to data/RF_sorted_data/TScore
Copied Teat-Score-9 to data/RF_sorted_data/TScore


In [132]:
reformat_coco_annots(TSCORE_OUT_DIR, TSCORE_YOLO_DIR)

Successfully converted Teat-Score-5 to yolov8 annotation format.
Successfully converted Teat-Score-7 to yolov8 annotation format.
Successfully converted Teat-Score-3 to yolov8 annotation format.
Successfully converted Teat-Score-9 to yolov8 annotation format.
Finished conversion of data/RF_sorted_data/TScore to yolov8 annotation format.


In [133]:
compare_file_counts(SOURCE_DATA_DIR, TSCORE_OUT_DIR, TSCORE_YOLO_DIR, main_class="Teat-Score")

Pre-aggregation: 117, Post-aggregation: 117, Post-conversion: 117


# Conversion of Cow Breed data

In [134]:
aggregate_class_folders(SOURCE_DATA_DIR, BREED_OUT_DIR, main_class='-Cow')

Copied Hallikar-Cow to data/RF_sorted_data/CowBreed
Copied HF-Crossbreed-Cow to data/RF_sorted_data/CowBreed
Copied Non-Descript-Breed-Cow to data/RF_sorted_data/CowBreed
Copied Jersey-Crossbreed-Cow to data/RF_sorted_data/CowBreed
Copied Gir-Cow to data/RF_sorted_data/CowBreed


In [135]:
reformat_coco_annots(BREED_OUT_DIR, BREED_YOLO_DIR)

Successfully converted Hallikar-Cow to yolov8 annotation format.
Successfully converted HF-Crossbreed-Cow to yolov8 annotation format.
Successfully converted Non-Descript-Breed-Cow to yolov8 annotation format.
Successfully converted Jersey-Crossbreed-Cow to yolov8 annotation format.
Successfully converted Gir-Cow to yolov8 annotation format.
Finished conversion of data/RF_sorted_data/CowBreed to yolov8 annotation format.


In [136]:
compare_file_counts(SOURCE_DATA_DIR, BREED_OUT_DIR, BREED_YOLO_DIR, main_class='Cow')

Pre-aggregation: 105, Post-aggregation: 105, Post-conversion: 105
